In [1]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import pandas as pd

import mlflow
from mlflow.models import infer_signature

2024-04-06 09:39:19.866058: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-06 09:39:20.918685: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-06 09:39:25.628966: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 09:39:36.324396: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Load data

In [2]:
mnist = tf.keras.datasets.mnist

In [3]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

### Tried this
# x_train = np.array(np.reshape(x_train, (-1,28*28)))
# x_test = np.array(np.reshape(x_test, (-1,28*28)))

In [4]:
# Normalize if necessary

x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

# Train!

In [5]:
model = tf.keras.models.Sequential()

# input layer
# Tried without it - used reshape instead.
model.add(tf.keras.layers.Flatten())

# two hidden dense layers 
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))

# output layer:
model.add(tf.keras.layers.Dense(10, activation='softmax'))

traininng_info = "Basic TensorFlow MNIST model"

params = dict(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    # loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

model.compile(**params)

# train the model
model.fit(x_train, y_train, epochs=3)

Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8676 - loss: 0.4689
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9649 - loss: 0.1167
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9772 - loss: 0.0735


In [6]:
val_loss, val_acc = model.evaluate(x_test, y_test)  # evaluate the out of sample data with model
(
   val_loss,  # model's loss (error)
   val_acc  # model's accuracy
)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9669 - loss: 0.1053


(0.09436739236116409, 0.9700000286102295)

# MLFlow

In [10]:
mlflow.set_experiment("mnist-test-02")

2024/04/06 09:46:37 INFO mlflow.tracking.fluent: Experiment with name 'mnist-test-02' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-artifacts/4', creation_time=1712396797797, experiment_id='4', last_update_time=1712396797797, lifecycle_stage='active', name='mnist-test-02', tags={}>

In [8]:
import os
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'
os.environ['AWS_ACCESS_KEY_ID'] = '74cvoBegkXBHT7jJ'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'RnCFp2DyrinPOMF8jtollC5fqvBiHcUY'

artifact_uri = "s3://mlflow-artifacts"

mlflow.set_tracking_uri(uri="http://mlflow:5000")

In [11]:
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", val_acc)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", traininng_info)

    # Infer the model signature
    # signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.tensorflow.log_model(
        model=model,
        artifact_path="mnist_model-test1",
        #signature=signature,
        input_example=x_train,
        registered_model_name="tracking-test",
    )
    
    model_uri = model_info.model_uri

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step


2024/04/06 09:53:28 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp5iqamknw/model, flavor: tensorflow). Fall back to return ['tensorflow==2.16.1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
Registered model 'tracking-test' already exists. Creating a new version of this model...
2024/04/06 09:54:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-test, version 13
Created version '13' of model 'tracking-test'.


In [22]:
    # predictions = model.predict(x_test)

    result = mlflow.evaluate(
        model,
        # Tried different variants of reshaping and type transformation there -
        # it didn't help
        data=x_test,
        targets=np.array(y_test),
        # predictions=predictions,
        model_type="classifier",
        # evaluators=["default"],
        # evaluator_config={"output_probabilities": True}
    )

MlflowException: If the `data` argument is a numpy array, it must be a 2-dimensional array, with the second dimension representing the number of features. If the `data` argument is a list, each of its elements must be a feature array of the numpy array or list, and all elements must have the same length.

---

# Predictions!

In [29]:
# import mlflow
logged_model = 'runs:/b9e7d7bf161949e09773275c0c2739fb/mnist_model-test1'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

predictions = loaded_model.predict(x_test)
print(predictions)

TypeError: <class 'keras.src.models.sequential.Sequential'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras', 'class_name': 'Sequential', 'config': {'name': 'sequential_3', 'trainable': True, 'dtype': 'float32', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [32, 28, 28], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_1'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'Flatten', 'config': {'name': 'flatten_3', 'trainable': True, 'dtype': 'float32', 'data_format': 'channels_last'}, 'registered_name': None, 'build_config': {'input_shape': [32, 28, 28]}}, {'module': 'keras.layers', 'class_name': 'Dense', 'config': {'name': 'dense_9', 'trainable': True, 'dtype': 'float32', 'units': 128, 'activation': 'relu', 'use_bias': Tru